In [1]:
import nltk
import random
import pandas as pd
import re

#importieren der Trainigsdaten---nltk stellt tweets dafür bereit
from nltk.corpus import twitter_samples

In [2]:
pos_twee = twitter_samples.strings('positive_tweets.json')
neg_twee = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')


In [3]:
print(pos_twee[0])

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [4]:
print(len(text))
print(len(pos_twee))
print(len(neg_twee))

20000
5000
5000


In [5]:
df = pd.DataFrame (pos_twee,columns=['text'])
df['Value'] = 1
df2 = pd.DataFrame (neg_twee,columns=['text'])
df2['Value'] = -1

In [6]:
df

,text,Value
0,#FollowFriday @France_Inte @PKuchly57 @Milipol...,1
1,@Lamb2ja Hey James! How odd :/ Please call our...,1
2,@DespiteOfficial we had a listen last night :)...,1
3,@97sides CONGRATS :),1
4,yeaaaah yippppy!!! my accnt verified rqst has...,1
...,...,...
4995,"@chriswiggin3 Chris, that's great to hear :) D...",1
4996,@RachelLiskeard Thanks for the shout-out :) It...,1
4997,@side556 Hey! :) Long time no talk...,1
4998,@staybubbly69 as Matt would say. WELCOME TO AD...,1


In [7]:
df2

,text,Value
0,hopeless for tmr :(,-1
1,Everything in the kids section of IKEA is so c...,-1
2,@Hegelbon That heart sliding into the waste ba...,-1
3,"“@ketchBurning: I hate Japanese call him ""bani...",-1
4,"Dang starting next week I have ""work"" :(",-1
...,...,...
4995,I wanna change my avi but uSanele :(,-1
4996,MY PUPPY BROKE HER FOOT :(,-1
4997,where's all the jaebum baby pictures :((,-1
4998,But but Mr Ahmad Maslan cooks too :( https://t...,-1


In [8]:
df=df.append(df2, ignore_index = True)

In [9]:
df = df.sample(frac=1).reset_index(drop=True) #msichen der positiven und negativen tweets


In [10]:
df


,text,Value
0,@thejcenaligway I still have to get a paper fr...,-1
1,Just in case Bae acts brand new :)💘,1
2,NOT AN APOLOGY ME ENCANTA VALE OSEA BEA :-(,-1
3,ANGRY GRANDPA PLAYS MINECRAFT (ANIMATION) #Min...,1
4,I so want to just be at home. I'm not eating p...,-1
...,...,...
9995,@mikecom17 Oh that's not good Linda :( Have yo...,-1
9996,"Yes, Pearson had that reputation. Does Nikkei ...",1
9997,"I :)""@Kreizi_: Who wants a #FF S/O??""",1
9998,@mcbride_alex sad times im gettin ready for a ...,-1


# bereinigen der Daten

In [11]:
#bereinigen der Tweets
remove_rt = lambda x: re.sub("(RT @\w+: )|(@\w+):"," ",x) #herausfiltern der usernamen und RT
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|[\d\.]|(\w+:\/\/\S+)|(\n|\t|\')"," ",x) #herausfiltern markierter Usernamen 
htp = lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ",x) #entfernen von links, sonderzeichen, punkten

df["text"] = df.text.map(remove_rt).map(rt).map(htp)
df["text"] = df.text.str.lower()
df.text.head(10)

0      i still have to get a paper from engg then g...
1                  just in case bae acts brand new :)💘
2          not an apology me encanta vale osea bea :-(
3    angry grandpa plays minecraft (animation) #min...
4    i so want to just be at home  i m not eating p...
5      ahaha i m laying on my bed like whyyyyyy  th...
6    snapchat : tammirossm #snapchat #kikgirl #kikc...
7    sketchbook art by love wilde drawing of hair i...
8                    follow   &amp;   follow u back :)
9                  i just want to look good for you :)
Name: text, dtype: object

In [12]:
#bereinigen des textes von den Emojis
def deEmojify(text):
    regrex_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'',text)

df["text"] = df.text.map(deEmojify)

df.text.head(5)

0      i still have to get a paper from engg then g...
1                   just in case bae acts brand new :)
2          not an apology me encanta vale osea bea :-(
3    angry grandpa plays minecraft (animation) #min...
4    i so want to just be at home  i m not eating p...
Name: text, dtype: object

In [13]:
data = list(df["text"])
labels = list(df["Value"])

In [14]:
data[:5]

['  i still have to get a paper from engg then go back to cmc then our before maging okay :((',
 'just in case bae acts brand new :)',
 'not an apology me encanta vale osea bea :-(',
 'angry grandpa plays minecraft (animation) #minecraft   ← click aqui :d  ',
 'i so want to just be at home  i m not eating properly and i m sleeping just to pass the time  i m miserable :(']

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

In [16]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

In [17]:
x_train[:10]

[' _   dale send a billion kiss to her :) have a magical day!',
 '    my main man jesus just looking out for me :)',
 '@_rossaholic thank you, donna!  it was uneventful but fine  :)',
 'tempted to eat this whole pack of oreos  need to find the strength not to do so  :(',
 '  thanks for the mention! honored to be included there among some great blogs :) have a great weekend! sharing now x',
 'so upset i m not going in november man, i hope there s going to be a next time in the uk :(',
 '  thank you nic!!!!!!!!! :)) hang soon sa taft ahh!!',
 '  all ice cream vans are busy in bristol :(',
 'wft   can t watch the awesome replay!! :-(  ',
 'thank you! :d  ']

In [18]:
vectorizer = TfidfVectorizer()

In [19]:
vectors = vectorizer.fit_transform(data)


vectors_train = vectorizer.transform(x_train)

In [20]:
model = MultinomialNB(alpha=.01)
model = model.fit(vectors_train, y_train)

In [21]:
vectors_test = vectorizer.transform(x_test)

# Auswertung des Models

In [25]:
from sklearn.metrics import classification_report

print(classification_report(y_test, model.predict(vectors_test), digits=4))

              precision    recall  f1-score   support

          -1     0.6888    0.7880    0.7351      1604
           1     0.7679    0.6633    0.7118      1696

    accuracy                         0.7239      3300
   macro avg     0.7284    0.7257    0.7234      3300
weighted avg     0.7295    0.7239    0.7231      3300



# Anreichern der Tweets

In [85]:
#einlesen der Daten
data_tweets = pd.read_json("./cryptodoge.json", lines=True)
df_tweets=data_tweets


In [86]:
#bereinigen der Tweets
remove_rt = lambda x: re.sub("(RT @\w+: )|(@\w+):"," ",x) #herausfiltern der usernamen und RT
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|[\d\.]|(\w+:\/\/\S+)|(\n|\t|\')"," ",x) #herausfiltern markierter Usernamen 
htp = lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ",x) #entfernen von links, sonderzeichen, punkten

df_tweets["text"] = df.text.map(remove_rt).map(rt).map(htp)
df_tweets["text"] = df.text.str.lower()
df_tweets.text.head(10)

#bereinigen des textes von den Emojis
def deEmojify(text):
    regrex_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'',text)

df_tweets["text"] = df_tweets.text.map(deEmojify)

In [87]:
df_tweets[:4]

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,display_text_range,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities
0,2021-06-28 16:25:33+00:00,1409548535272407047,1409548535272407040,i still have to get a paper from engg then g...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:25:33.611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-06-28 16:25:33+00:00,1409548536740552712,1409548536740552704,just in case bae acts brand new :),"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:25:33.961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-06-28 16:25:34+00:00,1409548538988609537,1409548538988609536,not an apology me encanta vale osea bea :-(,"<a href=""http://twitter.com/download/android"" ...",False,1.409546e+18,1.409546e+18,8.055323e+17,8.055323e+17,...,en,2021-06-28 16:25:34.497,"[16, 51]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-06-28 16:25:34+00:00,1409548540234194947,1409548540234194944,angry grandpa plays minecraft (animation) #min...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.406900e+18,1.406900e+18,1.074695e+18,1.074695e+18,...,en,2021-06-28 16:25:34.794,"[13, 140]",{'full_text': '@BitcoinAssn The most powerful ...,0.0,NaN,NaN,NaN,NaN,NaN


In [88]:
clean_data_list = list(df_tweets["text"])

In [89]:
clean_data_list[:5]

['  i still have to get a paper from engg then go back to cmc then our before maging okay :((',
 'just in case bae acts brand new :)',
 'not an apology me encanta vale osea bea :-(',
 'angry grandpa plays minecraft (animation) #minecraft   ← click aqui :d  ',
 'i so want to just be at home  i m not eating properly and i m sleeping just to pass the time  i m miserable :(']

In [90]:
#kalkulieren der Wahrscheinlichkeiten des jeweiligen Textes

In [91]:
one_test = vectorizer.transform(list(clean_data_list))

In [92]:
prob_single=model.predict_proba(one_test)
prob_single

array([[0.68235819, 0.31764181],
       [0.00517117, 0.99482883],
       [0.99855744, 0.00144256],
       ...,
       [0.15434131, 0.84565869],
       [0.52146387, 0.47853613],
       [0.87516354, 0.12483646]])

In [93]:


for i in prob_single:
    data_tweets["neg_prob"]=i[0]
    data_tweets["pos_prob"]=i[1]

In [94]:
data_tweets

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,display_text_range,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,neg_prob,pos_prob
0,2021-06-28 16:25:33+00:00,1409548535272407047,1409548535272407040,i still have to get a paper from engg then g...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
1,2021-06-28 16:25:33+00:00,1409548536740552712,1409548536740552704,just in case bae acts brand new :),"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
2,2021-06-28 16:25:34+00:00,1409548538988609537,1409548538988609536,not an apology me encanta vale osea bea :-(,"<a href=""http://twitter.com/download/android"" ...",False,1.409546e+18,1.409546e+18,8.055323e+17,8.055323e+17,...,"[16, 51]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
3,2021-06-28 16:25:34+00:00,1409548540234194947,1409548540234194944,angry grandpa plays minecraft (animation) #min...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.406900e+18,1.406900e+18,1.074695e+18,1.074695e+18,...,"[13, 140]",{'full_text': '@BitcoinAssn The most powerful ...,0.0,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
4,2021-06-28 16:25:34+00:00,1409548540494245896,1409548540494245888,i so want to just be at home i m not eating p...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,2021-06-28 16:36:26+00:00,1409551275440619527,1409551275440619520,"they don’t love me,they know me when they need...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
1003,2021-06-28 16:36:27+00:00,1409551275893665795,1409551275893665792,thanks :),"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
1004,2021-06-28 16:36:28+00:00,1409551282596057090,1409551282596057088,_decker whats ur height??? :)),"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
1005,2021-06-28 16:36:31+00:00,1409551296525332485,1409551296525332480,just americano :),"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.314213e+18,1.314213e+18,9.856861e+17,9.856861e+17,...,"[16, 140]",{'full_text': '@BinanceAcademy The most powerf...,0.0,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836


In [97]:
for index, neg in data_tweets["neg_prob"].iteritems():
    


    if neg > 0.55 :
        df.loc[index, "sentiment"] = "negative"

    elif neg < 0.45:
        df.loc[index, "sentiment"] = "positive"

    else:
        df.loc[index, "sentiment"] = "neutral"
        

In [98]:
data_tweets

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,display_text_range,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,neg_prob,pos_prob
0,2021-06-28 16:25:33+00:00,1409548535272407047,1409548535272407040,i still have to get a paper from engg then g...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
1,2021-06-28 16:25:33+00:00,1409548536740552712,1409548536740552704,just in case bae acts brand new :),"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
2,2021-06-28 16:25:34+00:00,1409548538988609537,1409548538988609536,not an apology me encanta vale osea bea :-(,"<a href=""http://twitter.com/download/android"" ...",False,1.409546e+18,1.409546e+18,8.055323e+17,8.055323e+17,...,"[16, 51]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
3,2021-06-28 16:25:34+00:00,1409548540234194947,1409548540234194944,angry grandpa plays minecraft (animation) #min...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.406900e+18,1.406900e+18,1.074695e+18,1.074695e+18,...,"[13, 140]",{'full_text': '@BitcoinAssn The most powerful ...,0.0,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
4,2021-06-28 16:25:34+00:00,1409548540494245896,1409548540494245888,i so want to just be at home i m not eating p...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,2021-06-28 16:36:26+00:00,1409551275440619527,1409551275440619520,"they don’t love me,they know me when they need...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
1003,2021-06-28 16:36:27+00:00,1409551275893665795,1409551275893665792,thanks :),"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
1004,2021-06-28 16:36:28+00:00,1409551282596057090,1409551282596057088,_decker whats ur height??? :)),"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
1005,2021-06-28 16:36:31+00:00,1409551296525332485,1409551296525332480,just americano :),"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.314213e+18,1.314213e+18,9.856861e+17,9.856861e+17,...,"[16, 140]",{'full_text': '@BinanceAcademy The most powerf...,0.0,NaN,NaN,NaN,NaN,NaN,0.875164,0.124836
